## Setup Models

In [ ]:
## Add my code to setup NaviGator connection

import openai
import os

base_url = os.environ.get('NAVIGATOR_BASE_URL')
OPENAI_API_KEY = os.environ.get('NAVIGATOR_GPT')
EMBEDDING_KEY = os.environ.get('NAVIGATOR_HIPERGATOR_MODELS')

# Set the environment variable
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Check list of available models
client = openai.OpenAI(
    api_key=OPENAI_API_KEY,
    base_url=base_url
)

response = client.models.list()
 
print(response)

# Print available models in better format
print('For the main OPENAI_API_KEY')
for model in response:
    print(model.id)

# We need an embedding model that that main key can't access for some reason.
# We'll set up the OPENAI_API_KEY environment variable for access to GPT-4o
# and use api_key=EMBEDDING_KEY for the embedding model.
client = openai.OpenAI(
    api_key=EMBEDDING_KEY,
    base_url=base_url
)

response = client.models.list()

print('For the EMBEDDING_KEY')
for model in response:
    print(model.id)


SyncPage[Model](data=[Model(id='mistral-7b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='command-r', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-4o-mini', created=1677610602, object='model', owned_by='openai'), Model(id='nim-mistral-7b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='llama-3.1-70b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='claude-3.5-sonnet', created=1677610602, object='model', owned_by='openai'), Model(id='gemini-1.0-pro', created=1677610602, object='model', owned_by='openai'), Model(id='nim-llama3.1-8b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='gemini-1.5-flash', created=1677610602, object='model', owned_by='openai'), Model(id='command-r-plus', created=1677610602, object='model', owned_by='openai'), Model(id='mistral-large', created=1677610602, object='model', owned_by='openai'), Model(id='codestral-22b', created=167

In [19]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import os

#Setup the environment variables
#os.environ["AZURE_OPENAI_API_KEY"]="4e4ab31800a64ae892cbb768fe28c0fc"
#os.environ["AZURE_OPENAI_ENDPOINT"]="https://agentic-ai-course-kumaran.openai.azure.com/"

#Setup the LLM
model = ChatOpenAI(
    model="gpt-4o",
    base_url=base_url
)

#Setup the Embedding
embedding = OpenAIEmbeddings(
    api_key=EMBEDDING_KEY,
    model="nomic-embed-text-v1.5",
    base_url=base_url
)

## 03.02. Add  Product Pricing function tool

In [20]:
import pandas as pd
from langchain_core.tools import tool

#Load the laptop product pricing CSV into a Pandas dataframe.
product_pricing_df = pd.read_csv("data/Laptop pricing.csv")
print(product_pricing_df)

@tool
def get_laptop_price(laptop_name:str) -> int :
    """
    This function returns the price of a laptop, given its name as input.
    It performs a substring match between the input name and the laptop name.
    If a match is found, it returns the pricxe of the laptop.
    If there is NO match found, it returns -1
    """

    #Filter Dataframe for matching names
    match_records_df = product_pricing_df[
                        product_pricing_df["Name"].str.contains(
                                                "^" + laptop_name, case=False)
                        ]
    #Check if a record was found, if not return -1
    if len(match_records_df) == 0 :
        return -1
    else:
        return match_records_df["Price"].iloc[0]

#Test the tool. Before running the test, comment the @tool annotation
#print(get_laptop_price("alpha"))
#print(get_laptop_price("testing"))
    

            Name  Price  ShippingDays
0  AlphaBook Pro   1499             2
1     GammaAir X   1399             7
2  SpectraBook S   2499             7
3   OmegaPro G17   2199            14
4  NanoEdge Flex   1699             2


## 03.03. Add Product Features Retrieval Tool

In [21]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma 
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the product featuers document.
loader=PyPDFLoader("./data/Laptop product descriptions.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

#Create a vector store with Chroma
prod_feature_store = Chroma.from_documents(
    documents=splits, 
    embedding=embedding
)

get_product_features = create_retriever_tool(
    prod_feature_store.as_retriever(search_kwargs={"k": 1}),
    name="Get_Product_Features",
    description="""
    This store contains details about Laptops. It lists the available laptops
    and their features including CPU, memory, storage, design and advantages
    """
)

#Test the product feature store
#print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

In [22]:
print(prod_feature_store.as_retriever().invoke("Tell me about the AlphaBook Pro") )

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(metadata={'page': 0, 'source': './data/Laptop product descriptions.pdf'}, page_content='Fictional Laptop Descriptions\nAlphaBook Pro\nThe AlphaBook Pro is a sleek ultrabook with a 12th Gen Intel i7 processor, 16GB of DDR4 RAM,\nand a fast 1TB SSD. Ideal for professionals on the go, this laptop offers an impressive blend of\npower and portability.\nGammaAir X\nGammaAir X combines an AMD Ryzen 7 processor with 32GB of DDR4 memory and a 512GB\nNVMe SSD. Its thin and light form factor makes it perfect for users who need high performance in a\nportable design.\nSpectraBook S\nDesigned for power users, SpectraBook S features an Intel Core i9 processor, 64GB RAM, and a\nmassive 2TB SSD. This workstation-class laptop is perfect for intensive tasks like video editing and\n3D rendering.\nOmegaPro G17\nOmegaPro G17 is a gaming powerhouse with a Ryzen 9 5900HX CPU, 32GB RAM, and a 1TB\nSSD. Designed for gamers, it features a 17-inch display with a high refresh rate and powerful\ngraphics

## 03.04.Setup a Product QnA chatbot

In [23]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create a System prompt to provide a persona to the chatbot
system_prompt = SystemMessage("""
    You are professional chatbot that answers questions about laptops sold by your company.
    To answer questions about laptops, you will ONLY use the available tools and NOT your own memory.
    You will handle small talk and greetings by producing professional responses.
    """
)

#Create a list of tools available
tools = [get_laptop_price, get_product_features]

#Create memory across questions in a conversation (conversation memory)
checkpointer=MemorySaver()

#Create a Product QnA Agent. This is actual a graph in langGraph
product_QnA_agent=create_react_agent(
                                model=model, #LLM to use
                                tools=tools, #List of tools to use
                                state_modifier=system_prompt, #The system prompt
                                debug=False, #Debugging turned on if needed
                                checkpointer=checkpointer #For conversation memory
)


In [24]:
#Setup chatbot
import uuid
#To maintain memory, each request should be in the context of a thread.
#Each user conversation will use a separate thread ID
config = {"configurable": {"thread_id": uuid.uuid4()}}

#Test the agent with an input
inputs = {"messages":[
                HumanMessage("What are the features and pricing for GammaAir?")
            ]}

#Use streaming to print responses as the agent  does the work.
#This is an alternate way to stream agent responses without waiting for the agent to finish
for stream in product_QnA_agent.stream(inputs, config, stream_mode="values"):
    message=stream["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What are the features and pricing for GammaAir?
================================== Ai Message ==================================
Tool Calls:
  Get_Product_Features (call_7UOI9Q2DAUXaeyVIfzxXFC81)
 Call ID: call_7UOI9Q2DAUXaeyVIfzxXFC81
  Args:
    query: GammaAir
  get_laptop_price (call_IsS3NWjrYRfnbm4XnwpYfRUg)
 Call ID: call_IsS3NWjrYRfnbm4XnwpYfRUg
  Args:
    laptop_name: GammaAir
================================= Tool Message =================================
Name: get_laptop_price

1399
================================== Ai Message ==================================

### GammaAir X
**Features:**
- **Processor:** AMD Ryzen 7
- **Memory:** 32GB DDR4
- **Storage:** 512GB NVMe SSD
- **Design:** Thin and light form factor, perfect for high performance in a portable design.

**Price:** $1399


## 03.05. Execute the Product QnA Chatbot

In [25]:
import uuid
#Send a sequence of messages to chatbot and get its response
#This simulates the conversation between the user and the Agentic chatbot
user_inputs = [
    "Hello",
    "I am looking to buy a laptop",
    "Give me a list of available laptop names",
    "Tell me about the features of  SpectraBook",
    "How much does it cost?",
    "Give me similar information about OmegaPro",
    "What info do you have on AcmeRight ?",
    "Thanks for the help"
]

#Create a new thread
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

for input in user_inputs:
    print(f"----------------------------------------\nUSER : {input}")
    #Format the user message
    user_message = {"messages":[HumanMessage(input)]}
    #Get response from the agent
    ai_response = product_QnA_agent.invoke(user_message,config=config)
    #Print the response
    print(f"AGENT : {ai_response['messages'][-1].content}")
    

----------------------------------------
USER : Hello
AGENT : Hello! How can I assist you today?
----------------------------------------
USER : I am looking to buy a laptop
AGENT : Great! I'd be happy to help you find a laptop that suits your needs. Do you have any specific requirements or preferences for the laptop, such as the type of CPU, amount of memory, storage capacity, or any other features you're looking for?
----------------------------------------
USER : Give me a list of available laptop names
AGENT : Here are the available laptop models:

1. **AlphaBook Pro**
2. **GammaAir X**
3. **SpectraBook S**
4. **OmegaPro G17**
5. **NanoEdge Flex**

Do any of these models catch your interest, or would you like more details on a specific one?
----------------------------------------
USER : Tell me about the features of  SpectraBook
AGENT : The **SpectraBook S** is designed for power users and comes with the following features:

- **Processor**: Intel Core i9
- **Memory**: 64GB RAM
- 

In [26]:
#conversation memory by user
def execute_prompt(user, config, prompt):
    inputs = {"messages":[("user",prompt)]}
    ai_response = product_QnA_agent.invoke(inputs,config=config)
    print(f"\n{user}: {ai_response['messages'][-1].content}")

#Create different session threads for 2 users
config_1 = {"configurable": {"thread_id": str(uuid.uuid4())}}
config_2 = {"configurable": {"thread_id": str(uuid.uuid4())}}

#Test both threads
execute_prompt("USER 1", config_1, "Tell me about the features of  SpectraBook")
execute_prompt("USER 2", config_2, "Tell me about the features of  GammaAir")
execute_prompt("USER 1", config_1, "What is its price ?")
execute_prompt("USER 2", config_2, "What is its price ?")




USER 1: The **SpectraBook S** is designed for power users and comes with the following features:
- **Processor:** Intel Core i9
- **Memory:** 64GB RAM
- **Storage:** 2TB SSD

This workstation-class laptop is ideal for intensive tasks such as video editing and 3D rendering.

USER 2: The **GammaAir X** laptop has the following features:

- **Processor**: AMD Ryzen 7
- **Memory**: 32GB of DDR4 RAM
- **Storage**: 512GB NVMe SSD
- **Design**: Thin and light form factor, making it highly portable
- **Ideal For**: Users who need high performance in a portable design

If you need more information or have any other questions, feel free to ask!

USER 1: The price of the SpectraBook S is $2499.

USER 2: The price of the GammaAir X laptop is $1399. If you have any other questions or need further assistance, feel free to ask!
